<a href="https://colab.research.google.com/github/francoalvaa/Music-System-Recommendation/blob/main/Extract%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installs

In [1]:
!curl https://api.discogs.com/releases/249504 --user-agent "FooBarApp/3.0"


{"id": 249504, "status": "Accepted", "year": 1987, "resource_url": "https://api.discogs.com/releases/249504", "uri": "https://www.discogs.com/release/249504-Rick-Astley-Never-Gonna-Give-You-Up", "artists": [{"name": "Rick Astley", "anv": "", "join": "", "role": "", "tracks": "", "id": 72872, "resource_url": "https://api.discogs.com/artists/72872"}], "artists_sort": "Rick Astley", "labels": [{"name": "RCA", "catno": "PB 41447", "entity_type": "1", "entity_type_name": "Label", "id": 895, "resource_url": "https://api.discogs.com/labels/895"}], "series": [], "companies": [{"name": "BMG Records (UK) Ltd.", "catno": "", "entity_type": "13", "entity_type_name": "Phonographic Copyright (p)", "id": 82835, "resource_url": "https://api.discogs.com/labels/82835"}, {"name": "BMG Records (UK) Ltd.", "catno": "", "entity_type": "14", "entity_type_name": "Copyright (c)", "id": 82835, "resource_url": "https://api.discogs.com/labels/82835"}, {"name": "Pete Waterman Ltd.", "catno": "", "entity_type": "13

In [ ]:
!pip install oauth2

#imports

In [ ]:
import requests

In [ ]:
#
import json
import sys

from urllib import request
from urllib.parse import parse_qsl
from urllib.parse import urlparse

import oauth2 as oauth

In [ ]:
import pandas as pd

#oauth

In [ ]:
consumer_key = 'jSmwcayelgGOTXCLbuco'
consumer_secret = 'FcaSGVGSNhyCRLTLmfvcTgNAXriKkzoI'

# The following oauth end-points are defined by discogs.com staff. These static endpoints
# are called at various stages of oauth handshaking.
request_token_url = 'https://api.discogs.com/oauth/request_token'
authorize_url = 'https://www.discogs.com/oauth/authorize'
access_token_url = 'https://api.discogs.com/oauth/access_token'

user_agent = 'recommedation_system'

# create oauth Consumer and Client objects using
consumer = oauth.Consumer(consumer_key, consumer_secret)
client = oauth.Client(consumer)

resp, content = client.request(request_token_url, 'POST', headers={'User-Agent': user_agent})

if resp['status'] != '200':
    sys.exit('Invalid response {0}.'.format(resp['status']))

request_token = dict(parse_qsl(content.decode('utf-8')))

print(' == Request Token == ')
print(f'    * oauth_token        = {request_token["oauth_token"]}')
print(f'    * oauth_token_secret = {request_token["oauth_token_secret"]}')
print()

# Authorize our newly received request_token against the discogs oauth endpoint.
# Prompt your user to "accept" the terms of your application. The application
# will act on behalf of their discogs.com account.
# If the user accepts, discogs displays a key to the user that is used for
# verification. The key is required in the 2nd phase of authentication.
print(f'Please browse to the following URL {authorize_url}?oauth_token={request_token["oauth_token"]}')


 == Request Token == 
    * oauth_token        = ByayWVHrnsGPcCFWMpQcpVKizqAHWTtjbCwGxzNA
    * oauth_token_secret = nrhZDQeytTiYLlfdXwIdOlAEpfyuxLzfCtLAHQJf

Please browse to the following URL https://www.discogs.com/oauth/authorize?oauth_token=ByayWVHrnsGPcCFWMpQcpVKizqAHWTtjbCwGxzNA


In [ ]:
# Waiting for user input
accepted = 'n'
while accepted.lower() == 'n':
    print()
    accepted = input(f'Have you authorized me at {authorize_url}?oauth_token={request_token["oauth_token"]} [y/n] :')

# request the verification token from the user.
oauth_verifier = input('Verification code : ')


Have you authorized me at https://www.discogs.com/oauth/authorize?oauth_token=ByayWVHrnsGPcCFWMpQcpVKizqAHWTtjbCwGxzNA [y/n] :y
Verification code : AaneAxANKT


In [ ]:
# Generate objects that pass the verification key with the oauth token and oauth
# secret to the discogs access_token_url
token = oauth.Token(request_token['oauth_token'], request_token['oauth_token_secret'])
token.set_verifier(oauth_verifier)
client = oauth.Client(consumer, token)

resp, content = client.request(access_token_url, 'POST', headers={'User-Agent': user_agent})

In [ ]:
access_token = dict(parse_qsl(content.decode('utf-8')))

print(' == Access Token ==')
print(f'    * oauth_token        = {access_token["oauth_token"]}')
print(f'    * oauth_token_secret = {access_token["oauth_token_secret"]}')
print(' Authentication complete. Future requests must be signed with the above tokens.')
print()

 == Access Token ==
    * oauth_token        = AIfKtJiYYLmoxTmrIBRkFRQoJQfhWEZmIoNSTDVx
    * oauth_token_secret = JytnePhoqxpifkatQmONJJMtXGfsbvJcQaHADPvK
 Authentication complete. Future requests must be signed with the above tokens.



In [ ]:
# along with the verified oauth token and oauth token for this user.
token = oauth.Token(key=access_token['oauth_token'],
        secret=access_token['oauth_token_secret'])
client = oauth.Client(consumer, token)


#Scraping

In [ ]:
all_songs = []


In [ ]:
page = 1
resp, content = client.request('https://api.discogs.com/database/search?genre=Electronic&page={page}&per_page=100',
        headers={'User-Agent': user_agent})

if resp['status'] != '200':
    sys.exit('Invalid API response {0}.'.format(resp['status']))

In [ ]:

releases = json.loads(content.decode('utf-8'))
for release in releases['results']:
    id = release.get("id")
    title = release.get("title")
    year = release.get("year", "Unknown")
    label = release.get("label", ["Unknown"])
    style = release.get("style", "Unknown")
    type = release.get("type")

    all_songs.append({'id':id, 'title': title, 'Year': year, 'label': label, 'style': style , 'type': type})

In [ ]:
df = pd.DataFrame(all_songs)
df

,id,title,Year,label,style,type
0,11520,Millsart / Jeff Mills - Inner Life / Medusa,1997,"[React, Axis, React, Axis Records, P.R. Record...",[Techno],release
1,316595,Casco Presents BWH* - Stop / Livin Up (Origina...,2004,[Radius Records],"[Italo-Disco, Electro, Tech House]",release
2,8334239,Various - Disco Mantras Vol. 1,2016,"[Mood Hut, Mood Hut Studios, The Exchange Viny...","[Ambient, Deep House, Disco, Downtempo, Africa...",release
3,554,FUSE* - Dimension Intrusion,1993,"[Warp Records, Artificial Intelligence, UTK, P...","[Abstract, Techno, Ambient, Acid, Downtempo, M...",release
4,1923,Bola - Soup,1998,"[Skam, Skam, Skam, Gramofonové Závody, The Bol...","[IDM, Ambient, Downtempo]",release
...,...,...,...,...,...,...
95,6924757,Luis CL - Untitled,2015,"[Superconscious Records, Juno, Superconscious ...","[House, Techno]",release
96,8193606,Ghibli - I'm Looking For You,2016,"[Dark Entries, Dark Entries Editions]",[Italo-Disco],release
97,443274,Messiah Corps - Wrong Side Of The Grave,2005,[Deathchant],[Hardcore],release
98,349019,Rhythm For Reasons - DJ SS Remixes,1995,"[Formation Records, Formation Studios, Formati...",[Jungle],release


In [ ]:
df_sorted = df.sort_values(by='id')
df_sorted

In [ ]:
import json
import pandas as pd
import sys
# Asumiendo que 'client' y 'user_agent' ya están definidos

all_songs = []
page = 1

while True:
    resp, content = client.request(f'https://api.discogs.com/database/search?genre=Electronic&page={page}&per_page=100',
                                   headers={'User-Agent': user_agent})

    if resp['status'] != '200':
        sys.exit(f'Invalid API response {resp["status"]}.')

    data = json.loads(content.decode('utf-8'))

    # Añadir canciones a la lista all_songs
    for release in data['results']:
        id = release.get("id")
        title = release.get("title")
        year = release.get("year", "Unknown")
        label = release.get("label", ["Unknown"])
        style = release.get("style", "Unknown")
        type = release.get("type")

        all_songs.append({'id': id, 'title': title, 'Year': year, 'label': label, 'style': style, 'type': type})

    # Verificar si hay una próxima página
    next_url = data.get('pagination', {}).get('urls', {}).get('next')
    if next_url:
        page += 1  # Incrementar el número de página
    else:
        break  # Salir del bucle si no hay más páginas

# Crear un DataFrame con todas las canciones
df = pd.DataFrame(all_songs)
print(df)

SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
